# Data Science Capstone Project - Week 3

<H2>PART 1 - Downloading and Cleaning the Data</H2>

In [ ]:
import pandas as pd
import wikipedia as wp

In [ ]:
# Read the wikipedia article into a dataframe
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.head()

In [ ]:
# Drop all unassigned boroughs - the assumption is made that unassigned boroughs have no useful neighbourhood info
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
df.shape

<h2>PART 2 - Getting the Coordinates</h2>

In [ ]:
import geocoder

In [ ]:
# Function for finding coordinates - Geocoder does not respond when this function (or the one provided as an example on coursera) is called; I instead had to use the csv file.
def get_coords(postal_code):
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.google(f'{postal_code}, Toronto, Ontario')
        lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    return latitude, longitude

In [ ]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

In [ ]:
# Merge the dataframe with the coordinates csv using the postal code as the key
df_merged = pd.merge(df, df_coords, on=['Postal Code'])

In [ ]:
df_merged.head()

<h2>PART 3 - Clustering Analysis</h2>

In [ ]:
%matplotlib inline
import folium

In [ ]:
#df = df_merged[df.Borough.str.contains("Toronto")]       # Remove any boroughs not including Toronto in the name (optional)
df = df_merged
df.reset_index(drop=True, inplace=True)
df.head()

<h3>Creating a map of neighbourhoods</h3>

In [ ]:
# Create map of Toronto
map_toronto = folium.Map(location=[43.6487, -79.38544], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto

<h3>Analysing the neighbourhoods using Foursquare</h3>

In [ ]:
# Defining foursquare credentials (using .env to hide)
import os
from dotenv import load_dotenv

load_dotenv('.env')

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
VERSION = '20180605'
LIMIT = 100

In [ ]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return relevant information
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                                )

In [ ]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighbourhood column back into dataframe and move to first column
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
# Group the rows by neighbourhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

<h3>Create a dataframe of most common venues</h3>

In [ ]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# New dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

<h3>Cluster Neighbourhoods</h3>

In [ ]:
from sklearn.cluster import KMeans

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


In [ ]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)

toronto_merged = toronto_merged.astype({"Cluster Labels":'int'})

toronto_merged.head()

<h2>Visualise the clusters</h2>

In [ ]:
# Create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.6487, -79.38544], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_clusters)

map_clusters